In [1]:
import pandas as pd
import numpy as np
import re
import json

In [2]:
df = pd.read_csv('http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/b270ef1a-c219-48b1-8399-b1458e225d39/download/14_iscrittixresidenzasedecorsogruppo.csv', delimiter=';')

decoder = pd.read_csv('http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/eae4ee94-0797-41d2-b007-bc6dad3ef3e2/download/07_iscrittixresidenza.csv', delimiter=';')

with open('decoders.json','r') as file:
    decoders = json.load(file)

cod_prov = decoders[0]
prov_cod = decoders[1]

df = df[df['AnnoA']=='2021/2022']

df['ResidenzaCOD'] = df['ResidenzaR'].apply(lambda x: prov_cod[x])

In [3]:
with open('cod_confini.json','r') as file:
    dec_conf = json.load(file)

In [4]:
filtro = [] 
for i,value in df.iterrows():
    prov = value['ResidenzaCOD']
    confini = dec_conf.get(str(prov),[])
    is_confinante = value['SedeP'] in confini 
    filtro.append(is_confinante)

In [5]:
pendolari = df[filtro]

In [6]:
pendolari['sede_ateneo'] = pendolari['SedeP'].apply(lambda x: cod_prov[str(x)])

/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
df['sede_ateneo'] = df['SedeP'].apply(lambda x: cod_prov[str(x)])

In [8]:
fuorisede = df[df['SedeP']!=df['ResidenzaCOD']]

In [9]:
fuorisede['sede_ateneo']=fuorisede['SedeP'].apply(lambda x: cod_prov[str(x)])

/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Uni con più pendolari

In [10]:
unis = []
rapps = []
for uni in df['AteneoNOME'].unique():
    n_p = pendolari[pendolari['AteneoNOME']==uni]['Isc'].sum() #numero pendolari
    n_t = df[df['AteneoNOME']==uni]['Isc'].sum() #numero totali
    r = n_p/n_t*100
    unis.append(uni)
    rapps.append(r)

In [11]:
uni_pendolari = pd.DataFrame({'uni':unis,
             'pendolari/totale':rapps}).sort_values(by='pendolari/totale', ascending=False)

In [107]:
uni_pendolari.to_csv('pendolari_per_uni.csv', index=False)

# Adesso cerchiamo province che attirano più pendolari!

In [137]:
provs = []
rapps = []
for prov in df['sede_ateneo'].unique():
    n_pendolari = pendolari[pendolari['sede_ateneo']==prov]['Isc'].sum()
    n_tot = df[df['sede_ateneo']==prov]['Isc'].sum()
    if n_tot > 1000:
        r = n_pendolari/n_tot*100
        provs.append(prov)
        rapps.append(r)

In [138]:
pendolari_tot = pd.DataFrame({
    'province':provs,
    'pendolari/totale':rapps
}).sort_values(by='pendolari/totale',ascending=False)

In [139]:
pendolari_tot['cod'] = pendolari_tot['province'].apply(lambda x: prov_cod[x])

In [140]:
pendolari_tot.to_csv('province_con_più_pendolari.csv',index=False)

# Uni con più fuorisede (NON REAL)

In [11]:
unis = []
rapps = []
for uni in df['AteneoNOME'].unique():
    n_f = fuorisede[fuorisede['AteneoNOME']==uni]['Isc'].sum()
    n_t = df[df['AteneoNOME']==uni]['Isc'].sum() #numero totali
    
    if n_f <0:
        raise Exception('Qualcosa non va')
    r = n_f/n_t*100
    unis.append(uni)
    rapps.append(r)

In [12]:
uni_fuorisede = pd.DataFrame({
    'Unis':unis,
    'real_fuorisede/totale':rapps,
}).sort_values(by='real_fuorisede/totale',ascending=False)

In [14]:
uni_fuorisede.to_csv('fuorisede_per_uni.csv',index=False)

# ora facciamo real fuorisede: cioè fuorisede generali - pendolari

# Uni con più real fuorisede

In [12]:
unis = []
rapps = []
for uni in df['AteneoNOME'].unique():
    n_p = pendolari[pendolari['AteneoNOME']==uni]['Isc'].sum() #numero pendolari
    n_f = fuorisede[fuorisede['AteneoNOME']==uni]['Isc'].sum()
    n_t = df[df['AteneoNOME']==uni]['Isc'].sum() #numero totali
    real_fuorisede = n_f - n_p
    
    if real_fuorisede <0:
        raise Exception('Qualcosa non va')
    r = real_fuorisede/n_t*100
    unis.append(uni)
    rapps.append(r)

In [13]:
uni_fuorisede = pd.DataFrame({
    'Unis':unis,
    'real_fuorisede/totale':rapps,
}).sort_values(by='real_fuorisede/totale',ascending=False)

In [14]:
uni_fuorisede

,Unis,real_fuorisede/totale
28,Milano Bocconi,75.168388
69,Roma Saint Camillus,65.252651
65,Milano San Raffaele,59.804766
26,Bologna,59.341509
54,Trieste,58.113183
...,...,...
14,Brescia,2.760883
8,Palermo,2.556252
7,Napoli Parthenope,2.491527
60,Basilicata,2.485896


In [201]:
uni_fuorisede.to_csv('real_fuorisede_per_uni.csv',index=False)

# Province con più real fuorisede

In [123]:
provs = []
rapps = []
for prov in df['ResidenzaR'].unique(): #mettiamo come elenco unique sempre province di residenza 
                                        #perché più completo
    n_pendolari = pendolari[pendolari['sede_ateneo']==prov]['Isc'].sum()
    n_fuorisede = fuorisede[fuorisede['sede_ateneo']==prov]['Isc'].sum()
    real_fuorisede = n_fuorisede-n_pendolari
    
    if real_fuorisede < 0:
        raise Exception('Qualcosa non va')
        
    n_tot = df[df['sede_ateneo']==prov]['Isc'].sum()
    #if n_tot > 1000:
    r = real_fuorisede/n_tot*100
    provs.append(prov)
    rapps.append(r)

/Users/sava/opt/anaconda3/envs/maps/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in long_scalars
  


In [124]:
arrivi = pd.DataFrame({
    'province':provs,
    'real_fuorisede/totale':rapps
}).sort_values(by='real_fuorisede/totale',ascending=False).fillna(0)

arrivi['cod'] = arrivi['province'].apply(lambda x: prov_cod[x])
arrivi.to_csv('province_accolgono_real_fuorisede.csv',index=False)

<h1> Ora real fuorisede ma partenze! </h1>

In [129]:
provs = []
rapps = []
for prov in df['ResidenzaR'].unique():
    n_pendolari = pendolari[pendolari['ResidenzaR']==prov]['Isc'].sum()
    n_fuorisede = fuorisede[fuorisede['ResidenzaR']==prov]['Isc'].sum()
    real_fuorisede = n_fuorisede-n_pendolari
    
    if real_fuorisede < 0:
        raise Exception('Qualcosa non va')
    n_tot = df[df['ResidenzaR']==prov]['Isc'].sum()
    #if n_tot > 5000:
    r = real_fuorisede/n_tot*100
    provs.append(prov)
    rapps.append(r)

In [130]:
partenze = pd.DataFrame({
    'province':provs,
    'real_fuorisede/totale':rapps
}).sort_values(by='real_fuorisede/totale',ascending=False)

partenze['cod'] = partenze['province'].apply(lambda x: prov_cod[str(x)])

partenze.to_csv('province_partono_real_fuorisede.csv',index=False)